# Spam classfication 

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import fasttext
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Reshape
from tensorflow.keras.callbacks import EarlyStopping


2023-11-05 12:40:20.218739: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-05 12:40:20.250113: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:40:20.506181: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-05 12:40:20.508835: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-05 12:40:22.623514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Dataset Location

In [2]:
dataset = '../data/Stress Detection.csv'

## Convert to dataframe

In [3]:
df = pd.read_csv(dataset)
df.head()

,subreddit,post_id,sentence_range,text,label,syntax_fk_grade,Stress Level
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,7.910952,-0.204167


# Replacing Column names

In [4]:
df = df.drop(['subreddit', 'post_id', 'sentence_range', 'syntax_fk_grade', 'Stress Level'], axis=1)

# Duplicate removal

In [5]:
df.drop_duplicates(inplace=True)

## Text Preprocessing
- Removing Stopwords
- Removing Punctuations

In [6]:
def cleanText(text):
    text = re.sub(r'RT @\w+: ', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text    

df['text'] = df['text'].apply(cleanText)
df.head()

,text,label
0,he said he had not felt that way before sugge...,1
1,hey there r assistance not sure if this is th...,0
2,my mom then hit me with the newspaper and it s...,1
3,until i met my new boyfriend he is amazing h...,1
4,october is domestic violence awareness month a...,1


## Deep Learning

In [7]:
cv = CountVectorizer()
X = cv.fit_transform(df['text'])
y = df['label']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train.toarray(), y_train, epochs=10, batch_size=32, validation_data=(X_test.toarray(), y_test))

model.save('../models/stressDetection.h5')
pickle.dump(cv, open('../models/cvstress.pkl', 'wb'))

2023-11-05 12:41:55.730784: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2023-11-05 12:41:55.730921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: n00b
2023-11-05 12:41:55.730959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: n00b
2023-11-05 12:41:55.731278: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.129.3
2023-11-05 12:41:55.731362: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.113.1
2023-11-05 12:41:55.731385: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.113.1 does not match DSO version 535.129.3 -- cannot find working devices in this configuration


Epoch 1/10
71/71 [==============================] - 3s 17ms/step - loss: 0.6025 - accuracy: 0.6860 - val_loss: 0.5044 - val_accuracy: 0.7717
Epoch 2/10
71/71 [==============================] - 1s 10ms/step - loss: 0.3459 - accuracy: 0.8729 - val_loss: 0.4691 - val_accuracy: 0.7770
Epoch 3/10
71/71 [==============================] - 1s 13ms/step - loss: 0.1647 - accuracy: 0.9535 - val_loss: 0.5329 - val_accuracy: 0.7575
Epoch 4/10
71/71 [==============================] - 1s 15ms/step - loss: 0.0729 - accuracy: 0.9907 - val_loss: 0.5648 - val_accuracy: 0.7646
Epoch 5/10
71/71 [==============================] - 1s 15ms/step - loss: 0.0321 - accuracy: 0.9956 - val_loss: 0.6283 - val_accuracy: 0.7575
Epoch 6/10
71/71 [==============================] - 1s 12ms/step - loss: 0.0156 - accuracy: 0.9987 - val_loss: 0.6922 - val_accuracy: 0.7593
Epoch 7/10
71/71 [==============================] - 1s 10ms/step - loss: 0.0117 - accuracy: 0.9987 - val_loss: 0.7072 - val_accuracy: 0.7646
Epoch 8/10
71